# Identifying Ethnicity in OpenSAFELY-TPP
This short report describes how ethnicity can be identified in the OpenSAFELY-TPP database, and the strengths and weaknesses of the methods. This is a living document that will be updated to reflect changes to the OpenSAFELY-TPP database and the patient records within.

## OpenSAFELY
OpenSAFELY is an analytics platform for conducting analyses on Electronic Health Records inside the secure environment where the records are held. This has multiple benefits: 

* We don't transport large volumes of potentially disclosive pseudonymised patient data outside of the secure environments for analysis
* Analyses can run in near real-time as records are ready for analysis as soon as they appear in the secure environment
* All infrastructure and analysis code is stored in GitHub repositories, which are open for security review, scientific review, and re-use

A key feature of OpenSAFELY is the use of study definitions, which are formal specifications of the datasets to be generated from the OpenSAFELY database. This takes care of much of the complex EHR data wrangling required to create a dataset in an analysis-ready format. It also creates a library of standardised and validated variable definitions that can be deployed consistently across multiple projects. 

The purpose of this report is to describe all such variables that relate to BMI, their relative strengths and weaknesses, in what scenarios they are best deployed. It will also describe potential future definitions that have not yet been implemented.

## Available Records
OpenSAFELY-TPP runs inside TPP’s data centre which contains the primary care records for all patients registered at practices using TPP’s SystmOne Clinical Information System. This data centre also imports external datasets from other sources, including A&E attendances and hospital admissions from NHS Digital’s Secondary Use Service, and death registrations from the ONS. More information on available data sources can be found within the OpenSAFELY documentation. 

In [2]:
from IPython.display import display, Markdown
from report_functions import *

import itertools
import matplotlib.gridspec as gridspec
import numpy as np
import pandas as pd
import seaborn as sns

from ebmdatalab import charts
from functools import reduce
from matplotlib import pyplot as plt

plt.rcParams.update({'figure.max_open_warning': 0})

In [3]:
### CONFIGURE OPTIONS HERE ###

# Import file
input_path = '../output/data/input.feather'

# Definitions
definitions = ['ethnicity_snomed_5', 'ethnicity_5']

# Dates
date_min = '2019-01-01'
date_max = '2019-12-31'
time_delta = 'M'

# Min/max range
min_range = 4
max_range = 200

# Null value – 0 or NA
null = 0

# Covariates
demographic_covariates = ['age_band', 'sex', 'region', 'imd']
clinical_covariates = ['dementia', 'diabetes', 'hypertension', 'learning_disability']

In [4]:
def redact_round_table(df_in):
    """Redacts counts <= 5 and rounds counts to nearest 5"""
    df_in = df_in.where(df_in > 5, np.nan).apply(lambda x: 5 * round(x/5))
    df_out = df_in.where(~df_in.isna(), '-')
    return df_out

In [5]:
df_import = pd.read_feather(input_path)

# Subset to relevant columns
df_clean = df_import[['patient_id'] + definitions + demographic_covariates + clinical_covariates]
df_clean['all']=True
# Set null values to nan
for definition in definitions: 
    df_clean.loc[df_clean[definition] == null, definition] = np.nan
    df_clean.loc[df_clean[definition] == "0", definition] = np.nan
 # Create order for categorical variables
for group in demographic_covariates + clinical_covariates:
    if df_clean[group].dtype.name == 'category':
        li_order = sorted(df_clean[group].dropna().unique().tolist())
        df_clean[group] = df_clean[group].cat.reorder_categories(li_order, ordered=True)
# Mark patients with value filled/missing for each definition
for definition in definitions:
    df_clean.loc[~df_clean[definition].isna(), definition+"_filled"] = 1
    df_clean.loc[df_clean[definition].isna(), definition+"_filled"] = 0
    df_clean.loc[df_clean[definition].isna(), definition+"_missing"] = 1
    df_clean.loc[~df_clean[definition].isna(), definition+"_missing"] = 0
    


In [6]:

def crosstab(dfr,idx,idy):
    cols = ["Attribute","No",idy, "%"]
    counts = pd.crosstab(dfr[idx], dfr[idy], normalize=False, dropna=False)
    percentages = (
        pd.crosstab(dfr[idx], dfr[idy], normalize="index", dropna=False)[1] * 100
    ).round(1)
    all_cols = pd.concat([counts, percentages], axis=1)
    all_cols=pd.DataFrame(all_cols).reset_index()
    all_cols.columns = cols
    return all_cols

statifiers=['all'] + demographic_covariates + clinical_covariates

def crosstab_cat(dfr,idx,idy):
    dfr[idy] = pd.factorize(dfr[idy], sort=True)[0] 
    counts = pd.crosstab(dfr[idx], dfr[idy], normalize=False, dropna=False)
    percentages = (
        pd.crosstab(dfr[idx], dfr[idy], normalize='index', dropna=False)*100
    ).round(1)
    n_perc = (counts.astype(str)+ " (" +percentages.astype(str) +")")
    all_cols=pd.DataFrame(n_perc)
    return all_cols

In [8]:

crosstabs = [crosstab(df_clean,v,definitions[0]+"_filled") for v in statifiers]
all_together = pd.concat(
    crosstabs,keys=statifiers,
)

all_together=all_together.reset_index()
all_together.rename(columns = {'level_0':'Covariate'}, inplace = True)
all_together=all_together.set_index(['Covariate','Attribute'])
all_together['N (%)'] = (all_together[f'{definitions[0]}_filled'].astype(str)+ " (" + all_together['%'].astype(str) +")")
all_together.drop(['level_1','No','%',f'{definitions[0]}_filled'], axis=1, inplace=True)

display("Number of patients with a valid " + definitions[0] +" code")
display(all_together)

'Number of patients with a valid ethnicity_snomed_5 code'

N (%)
Covariate           Attribute                           
all                 True                      681 (68.1)
age_band            0-19                       93 (73.2)
                    20-29                      90 (69.2)
                    30-39                      79 (66.4)
                    40-49                      76 (62.8)
                    50-59                      80 (66.7)
                    60-69                      90 (68.7)
                    70-79                      97 (70.8)
                    80+                        76 (66.1)
sex                 F                         332 (67.6)
                    M                         349 (68.6)
region              East Midlands              42 (63.6)
                    East of England            41 (66.1)
                    London                     87 (66.9)
                    North East                 44 (67.7)
                    North West                 45 (71.4)
                    South East                 94 (70.7)
                    West Midlands              46 (64.8)
                    Yorkshire and the Humber   41 (68.3)
imd                 100                       142 (66.7)
                    200                       145 (73.6)
                    300                       120 (69.0)
                    400                       148 (67.3)
                    500                       126 (64.3)
dementia            False                     676 (68.3)
                    True                        5 (50.0)
diabetes            False                     672 (67.9)
                    True                        9 (90.0)
hypertension        False                     679 (68.6)
                    True                        2 (20.0)
learning_disability False                     671 (67.8)
                    True                      10 (100.0)

### Count of Missings
Note: Should add the population N to this table. Do we want percentage to be computed from total population or total recorded?

In [10]:
crosstabs = [crosstab(df_clean,v,definitions[0]+"_missing") for v in statifiers]
all_together = pd.concat(
    crosstabs,keys=statifiers,
)

all_together=all_together.reset_index()
all_together.rename(columns = {'level_0':'Covariate'}, inplace = True)
all_together=all_together.set_index(['Covariate','Attribute'])
all_together['N (%)'] = (all_together[f'{definitions[0]}_missing'].astype(str)+ " (" + all_together['%'].astype(str) +")")
all_together.drop(['level_1','No','%',f'{definitions[0]}_missing'], axis=1, inplace=True)

display("Number of patients missing any " + definitions[0] +" code")
display(all_together)

'Number of patients missing any ethnicity_snomed_5 code'

N (%)
Covariate           Attribute                           
all                 True                      319 (31.9)
age_band            0-19                       34 (26.8)
                    20-29                      40 (30.8)
                    30-39                      40 (33.6)
                    40-49                      45 (37.2)
                    50-59                      40 (33.3)
                    60-69                      41 (31.3)
                    70-79                      40 (29.2)
                    80+                        39 (33.9)
sex                 F                         159 (32.4)
                    M                         160 (31.4)
region              East Midlands              24 (36.4)
                    East of England            21 (33.9)
                    London                     43 (33.1)
                    North East                 21 (32.3)
                    North West                 18 (28.6)
                    South East                 39 (29.3)
                    West Midlands              25 (35.2)
                    Yorkshire and the Humber   19 (31.7)
imd                 100                        71 (33.3)
                    200                        52 (26.4)
                    300                        54 (31.0)
                    400                        72 (32.7)
                    500                        70 (35.7)
dementia            False                     314 (31.7)
                    True                        5 (50.0)
diabetes            False                     318 (32.1)
                    True                        1 (10.0)
hypertension        False                     311 (31.4)
                    True                        8 (80.0)
learning_disability False                     319 (32.2)
                    True                         0 (0.0)

### Count of Patients

In [14]:
all_together_dict = {}
categories=df_clean[definitions[0]].unique()

## append tables for each definition
for definition in definitions:
    crosstabs_cat = [crosstab_cat(df_clean,v,definition) for v in statifiers]
    all_together_cat = pd.concat(
         crosstabs_cat,keys=statifiers,
     )
    all_together_cat=all_together_cat.rename(columns=str).reset_index()
    all_together_cat.rename(columns = {'level_0':'Covariate','level_1':'Attribute'}, inplace = True)
    all_together_dict[definition] = all_together_cat.set_index(['Covariate','Attribute'])
all_together_dict=pd.concat(all_together_dict,axis = 1)

# print(f"Counts per Category {definitions[0]}")
display(all_together_dict[definitions[0]])

Counts per Category ethnicity_snomed_5


ethnicity_snomed_5                                     0           1  \
Covariate           Attribute                                          
all                 True                      319 (31.9)  181 (18.1)   
age_band            0-19                       34 (26.8)   24 (18.9)   
                    20-29                      40 (30.8)   24 (18.5)   
                    30-39                      40 (33.6)   19 (16.0)   
                    40-49                      45 (37.2)   24 (19.8)   
                    50-59                      40 (33.3)   22 (18.3)   
                    60-69                      41 (31.3)   21 (16.0)   
                    70-79                      40 (29.2)   27 (19.7)   
                    80+                        39 (33.9)   20 (17.4)   
sex                 F                         159 (32.4)   92 (18.7)   
                    M                         160 (31.4)   89 (17.5)   
region              East Midlands              24 (36.4)   13 (19.7)   
                    East of England            21 (33.9)   17 (27.4)   
                    London                     43 (33.1)   18 (13.8)   
                    North East                 21 (32.3)   14 (21.5)   
                    North West                 18 (28.6)   10 (15.9)   
                    South East                 39 (29.3)   21 (15.8)   
                    West Midlands              25 (35.2)   10 (14.1)   
                    Yorkshire and the Humber   19 (31.7)   10 (16.7)   
imd                 100                        71 (33.3)   45 (21.1)   
                    200                        52 (26.4)   43 (21.8)   
                    300                        54 (31.0)   32 (18.4)   
                    400                        72 (32.7)   34 (15.5)   
                    500                        70 (35.7)   27 (13.8)   
dementia            False                     314 (31.7)  180 (18.2)   
                    True                        5 (50.0)    1 (10.0)   
diabetes            False                     318 (32.1)  180 (18.2)   
                    True                        1 (10.0)    1 (10.0)   
hypertension        False                     311 (31.4)  180 (18.2)   
                    True                        8 (80.0)    1 (10.0)   
learning_disability False                     319 (32.2)  178 (18.0)   
                    True                         0 (0.0)    3 (30.0)   

ethnicity_snomed_5                                     2           3  \
Covariate           Attribute                                          
all                 True                      107 (10.7)  180 (18.0)   
age_band            0-19                       15 (11.8)   27 (21.3)   
                    20-29                       10 (7.7)   26 (20.0)   
                    30-39                      16 (13.4)   19 (16.0)   
                    40-49                       11 (9.1)   17 (14.0)   
                    50-59                      12 (10.0)   21 (17.5)   
                    60-69                       11 (8.4)   25 (19.1)   
                    70-79                      18 (13.1)   23 (16.8)   
                    80+                        14 (12.2)   22 (19.1)   
sex                 F                          52 (10.6)   76 (15.5)   
                    M                          55 (10.8)  104 (20.4)   
region              East Midlands                4 (6.1)    8 (12.1)   
                    East of England              5 (8.1)   10 (16.1)   
                    London                     21 (16.2)   20 (15.4)   
                    North East                   6 (9.2)   14 (21.5)   
                    North West                   6 (9.5)   14 (22.2)   
                    South East                 18 (13.5)   26 (19.5)   
                    West Midlands                7 (9.9)   13 (18.3)   
                    Yorkshire and the Humber     4 (6.7)   14 (23.3)   
imd                 100                

In [13]:
#### resort columns to directly compare levels of definition (requires each defintion to have the same levels)
all_together_dict_slice = {}
categories=df_clean[definitions[0]].unique()
# create dict to rename columns
dict={}
for category in categories:
    for definition in definitions:
        # append dict
        dict.update({f"{definition}_{category}":f"{definition} {category}"})
        all_together_dict_slice[f"{definition}_{category}"] = all_together_dict[(definition,f"{category}")]
all_together_dict_slice=pd.concat(all_together_dict_slice,axis = 1)
all_together_dict_slice.rename(columns=dict, inplace=True)
# print(f" \n \n Counts per Category {definitions}")
display(all_together_dict_slice)


# categories=df_clean[definitions[0]].unique()
# for category in categories:
#     for definition in definitions:
#         df_clean2 = df_clean[(df_clean[definition] == category)]
#         crosstabs_cat = [crosstab_cat(df_clean2,v,definition) for v in statifiers]
#         all_together = pd.concat(
#              crosstabs_cat,keys=statifiers,
#          )
#         all_together=all_together.rename(columns=str).reset_index()
#         all_together.rename(columns = {'level_0':'Covariate','level_1':'Attribute','0':category}, inplace = True)
#         all_together_now[f"all_{definition}_{category}"] = all_together.set_index(['Covariate','Attribute'])

# all_together_now=pd.concat(all_together_now,axis = 1)
# display(all_together_now)


 
 
 Counts per Category ['ethnicity_snomed_5', 'ethnicity_5']


ethnicity_snomed_5 0  \
Covariate           Attribute                                       
all                 True                               181 (18.1)   
age_band            0-19                                34 (26.8)   
                    20-29                               40 (30.8)   
                    30-39                               40 (33.6)   
                    40-49                               45 (37.2)   
                    50-59                               40 (33.3)   
                    60-69                               41 (31.3)   
                    70-79                               40 (29.2)   
                    80+                                 39 (33.9)   
sex                 F                                  159 (32.4)   
                    M                                  160 (31.4)   
region              East Midlands                       24 (36.4)   
                    East of England                     21 (33.9)   
                    London                              43 (33.1)   
                    North East                          21 (32.3)   
                    North West                          18 (28.6)   
                    South East                          39 (29.3)   
                    West Midlands                       25 (35.2)   
                    Yorkshire and the Humber            19 (31.7)   
imd                 100                                 71 (33.3)   
                    200                                 52 (26.4)   
                    300                                 54 (31.0)   
                    400                                 72 (32.7)   
                    500                                 70 (35.7)   
dementia            False                              314 (31.7)   
                    True                                 5 (50.0)   
diabetes            False                              318 (32.1)   
                    True                                 1 (10.0)   
hypertension        False                              311 (31.4)   
                    True                                 8 (80.0)   
learning_disability False                              319 (32.2)   
                    True                                  0 (0.0)   

                                             ethnicity_5 0  \
Covariate           Attribute                                
all                 True                        134 (13.4)   
age_band            0-19                         30 (23.6)   
                    20-29                        32 (24.6)   
                    30-39                        32 (26.9)   
                    40-49                        40 (33.1)   
                    50-59                        26 (21.7)   
                    60-69                        26 (19.8)   
                    70-79                        36 (26.3)   
                    80+                          28 (24.3)   
sex                 F                           124 (25.3)   
                    M                           126 (24.8)   
region              East Midlands                16 (24.2)   
                    East of England              15 (24.2)   
                    London                       32 (24.6)   
                    North East                    9 (13.8)   
                    North West                   15 (23.8)   
                    South East                   32 (24.1)   
                    West Midlands                16 (22.5)   
                    Yorkshire and the Humber     14 (23.3)   
imd                 100                          54 (25.4)   
                    200                          45 (22.8)   
                    300                          33 (19.0)   
                    400                          63 (28.6)   
                    500                          55 (28.1)   
dementia            False                       248 (25.1)   
                    True

In [202]:
# redact_round_table(df_all_ct)

In [203]:
# # All with measurement
# li_missing = []
# for definition in definitions:
#     df_temp = df_clean[['patient_id', definition+'_missing']].drop_duplicates().dropna().set_index('patient_id')
#     li_missing.append(df_temp)
    
# df_temp3 = pd.concat(li_missing, axis=1)
# df_miss = pd.DataFrame(df_temp3.sum()).T
# df_miss['group'],df_miss['subgroup'] = ['population','all']
# df_miss = df_miss.set_index(['group','subgroup'])

# # By group
# li_group = []
# for group in demographic_covariates + clinical_covariates:
#     li_missing_group = []
#     for definition in definitions:
#         df_temp = df_clean[['patient_id', definition+'_missing', group]].drop_duplicates().dropna().reset_index(drop=True)
#         li_missing_group.append(df_temp)
#     df_reduce = reduce(lambda df1, df2: pd.merge(df1, df2,on=['patient_id',group],how='outer'), li_missing_group)
#     df_reduce2 = df_reduce.sort_values(by=group).drop(columns=['patient_id']).groupby(group).sum()
#     df_reduce2['group'] = group
#     li_group.append(df_reduce2)
# df_miss_group = pd.concat(li_group).reset_index().rename(columns={'index':'subgroup'}).set_index(['group','subgroup'])
# df_miss_ct = df_miss.append(df_miss_group)

In [204]:
# redact_round_table(df_miss_ct)

### Overlapping Definitions
Note: Heat map?? How to visualise?

In [205]:
# # Create columns for definition overlap
# df_temp2
# for def_pair in list(itertools.combinations(definitions, 2)):
#     df_temp2.loc[(~df_temp2[def_pair[0]+'_filled'].isna()) & (~df_temp2[def_pair[1]+'_filled'].isna()), def_pair[0] + "_" + def_pair[1]] = 1
# df_base = df_temp2[['derived_bmi_filled','computed_bmi_filled','recorded_bmi_filled']].sum()
# df_both = df_temp2[['derived_bmi_computed_bmi','computed_bmi_recorded_bmi','derived_bmi_recorded_bmi']].sum()

In [206]:
# df_temp2.sum()

### Number of Patients Over Time

In [207]:
# df_all_time = redact_round_table(df_clean[['date'] + definitions].groupby('date').count())
# df_all_time = df_all_time.stack().reset_index().rename(columns={'level_1':'variable',0:'value'})
# fig, ax = plt.subplots(figsize=(12, 8))
# fig.autofmt_xdate()
# sns.lineplot(x = 'date', y = 'value', hue='variable', data = df_all_time, ax=ax).set_title('New records by month')
# ax.legend().set_title('')

In [208]:
# for group in demographic_covariates + clinical_covariates:
#     for definition in definitions: 
#         df_time = redact_round_table(df_clean[['date',group,definition]].groupby(['date',group]).count()).reset_index()
#         df_time = df_time.replace('-',np.nan)
#         fig, ax = plt.subplots(figsize=(12, 8))
#         fig.autofmt_xdate()
#         sns.lineplot(x = 'date', y = definition, hue=group, data = df_time, ax=ax).set_title(f'{definition} recorded by {group} and month')
#         ax.legend().set_title('')